## 1. 必要なパッケージを読み込む

- Seurat: １細胞RNA-seqデータ解析の統合パッケージ
- tidyverse: テーブル形式のデータを解析するための統合パッケージ
    - モダンなデータサイエンスのための便利なパッケージ群


In [ ]:
# Command-01

library(Seurat)

In [ ]:
# Command-02

library(tidyverse)

## 2. シードの固定

描画結果などがランダムになる部分の計算を、制御するためのおまじない。

In [ ]:
# Command-03

set.seed(1234)

## 3. データ（遺伝子発現量行列）を読み込む

- このデータは、各行が遺伝子、各列が細胞を表してます
- つまり、
    - 行数＝遺伝子の数
    - 列数＝細胞の数
    - $i$ 行目、$j$ 列目の要素の数値は、$i$ 番目の遺伝子が $j$ 番目の細胞での発現量


In [ ]:
# Command-04

# （ちょっと時間がかかります）

df_counts <- read_tsv("data/GSM3173562_Lakshmipuram_NCBI_processeddata.txt", col_names=TRUE)

## 4. 読み込んだデータの「形」を確認する

- 「形」と読んでいるのは、行数、列数、どんな値が入っているかなど

In [ ]:
# Command-05

# 中身の確認 (最初の数行を出力する)

head(df_counts)

In [ ]:
# Command-06

# 行数

nrow(df_counts)

In [ ]:
# Command-07

# 列数

ncol(df_counts)

## 5. データを Seurat の形に変換する

In [ ]:
# Command-08

# データフレームを行列に変換する
df_counts %>% 
select(-GENE) %>% 
as.matrix() ->
mat_counts

# 最初の数行を確認する

head(mat_counts)

In [ ]:
# Command-09

# データフレーム df_counts の GENE という列の値を、行列 mat_counts の行の名前として代入する

rownames(mat_counts) <- df_counts$GENE

# 最初の数行を確認する (Command-08 の時と比較して、行に名前がついた)

head(mat_counts)

In [ ]:
# Command-10

# 作った行列の構造を確認する（出力の１行目は何行 x 何列か、３行目は行の名前 (row names)、4行目は列の名前 (column names) を表す）

str(mat_counts)

In [ ]:
# Command-11

# Seurat オブジェクトに変換する 
# counts のところに先ほど作った mat_counts を設定
# project のところはなんでもいい（ここでは　"planarian_2k" を設定)

planarian <-  CreateSeuratObject(counts = mat_counts, project = "planarian_2k")

In [ ]:
# Command-12

# 作った Seurat オブジェクト (planarian) を確認する 
# (features は遺伝子、samples は細胞と読み替えてください)

planarian

## 6: 品質の低い細胞をフィルターする

In [ ]:
# Command-13

# Violin plot という種類のグラフ
# 各点は細胞を表す
# 横幅が大きい値のところに点（＝細胞）が多く集中している

# - nFeature_RNA は「その細胞で、何個の遺伝子の発現がみられたか」
# - nCount_RNA は「その細胞での、各遺伝子のカウントの合計値」
# 例：遺伝子が５つの生き物を想定する、５つの遺伝子のカウントがそれぞれ[10, 15, 0, 23, 594] だった時、
#      nFeature_RNA は4, nFeature_RNAは 642 (=10+15+23+594) となる

VlnPlot(planarian, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2)

In [ ]:
# Command-14

# 散布図
# 点は細胞を表す
# 横軸は nCount_RNA
# 縦軸は nFeature_RNA

FeatureScatter(planarian, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

In [ ]:
# Command-15

# ここでは、RNAがあまり検出できていない細胞の情報を除きたい。
# そのために、RNAがある程度検出できている細胞を残すという操作をしている（混乱しないように）。

# `subset()` という関数は、特定の条件の細胞だけを選び取るために用いている。
# ここでは nFeature_RNA が200以上、かつ、nCount_RNA が 500 以上である細胞を選んでいる
# (`nFeature_RNA >= 200 & nCount_RNA >= 500` の部分で条件を指定している）


planarian <- subset(planarian, subset = nFeature_RNA >= 200 & nCount_RNA >= 500)

In [ ]:
# Command-16

# Command-12 では 2000 samples だったが、 1131 samples になっている。
# つまり、RNAがある程度検出できている細胞を残した（RNAがあまり検出できていない細胞の情報を除いた）。

planarian

In [ ]:
# Command-17

# Command-14 の時より0に近い細胞（点）がなくなっていることに注意

FeatureScatter(planarian, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

## 7. 発現量データを正規化する

- 細胞間で遺伝子発現量を比較できるように、正規化（Nomalization）を行う。
- ここでは、 `LogNormalize` という方法を使う
    - ある細胞 $j$ でのある遺伝子 $i$ の発現量を $x_{i,j}$  とする
    - scale.factor の値を $\alpha$ とする
    - まず、ある細胞 $j$ で feature (遺伝子) のカウントを合計する（この合計値を $S_j = \sum_i x_{i,j}$ とする)
    - 次に、ある細胞 $j$ での遺伝子のカウント $x_{i,j}$  を $S_j$ で割って $\alpha$ をかける ($\frac{x_{i,j}}{S_j}  \alpha $ )
    - さらに、1を足して、 logをとる ($y_{i,j} = \log(\frac{x_{i,j}}{S_j}  \alpha + 1)$)
    - この $y_{i,j}$ が正規化された発現量



In [ ]:
#  Command-18

# 遺伝子発現量を正規化している

planarian <- NormalizeData(planarian, normalization.method = "LogNormalize", scale.factor = 10000)

## 8. 高変動遺伝子（highly variabe genes) を抽出する

- 高変動遺伝子（highly variabe genes; HVG) は、細胞間で発現量の変動が大きい遺伝子のこと
- １細胞RNA-seqのデータは遺伝子が数万 (今のデータだと 51562 features) ある「高次元データ」です
- 高次元データで全ての変数（＝ここでは遺伝子）の値を用いて解析すると、計算時間がかかったり、ノイズの影響を受けたりする
- そこで、「遺伝子発現量が、ランダムなノイズによる変動よりも大きな細胞間変動を示している遺伝子」を抽出することで、細胞間での生物学的な変動を反映した遺伝子だけにしてデータ解析をしたい
- そのために、ここでは、どれが高変動遺伝子かを見つけます

In [ ]:
# Command-19

# どれが高変動遺伝子かを見つけます
# 変動が大きな 2000個の遺伝子を探す (nfeatures でってい)

planarian <- FindVariableFeatures(planarian, selection.method = "vst", nfeatures = 2000)

In [ ]:
# Command-20

# トップ10の高変動遺伝子を抽出する

top10 <- head(VariableFeatures(planarian), 10)

print(top10)

In [ ]:
# Command-21

# 高変動遺伝子の
# 点は遺伝子を表す
# 横軸は、遺伝子発現量を細胞間で平均をとった値
# 縦軸は、遺伝子発現量の細胞間での分散（を標準化したもの）

# 上にある点（＝遺伝子）は変動が大きい。→高変動遺伝子と考える
# 高変動遺伝子と判定された遺伝子は赤くなっている

plot1 <- VariableFeaturePlot(planarian)
plot1

# 同じグラフで、変動が大きい遺伝子について、遺伝子名を表示している

LabelPoints(plot = plot1, points = top10, repel = TRUE)

## 9. データをスケーリングする

- このあと行う主成分分析などでは、各遺伝子の発現量のスケールが揃っていていることが想定されている
- しかし、現状では、低発現の遺伝子と高発現の遺伝子では、発現量に1000~10000倍の差がある
- そこで、遺伝子間でのスケールを揃えるために、スケーリングを行う
- 具体的には
    - 遺伝子ごとに、合計が０になるようにし（センタリング; centering) 、その上で、分散が1になるようにする
- スケーリングの、細胞 $i$, 遺伝子 $j$ の発現量は $y^\prime_{i,j}$ と考えられる


In [ ]:
# Command-22

#  スケーリングの対象の遺伝子を選ぶ (全遺伝子を使う)

all.genes <- rownames(planarian)

# `all.genes` の要素数（スケーリングの対象の遺伝子の数）を念のため確認
print(length(all.genes))

# 遺伝子発現量をスケーリングする
# (少し時間がかかります)

planarian <- ScaleData(planarian, features = all.genes)

## 10. PCA（主成分分析）を用いて次元削減を行う

- 主成分分析
    - Principal component analysis (PCA) 
- PCAでは、主成分（PC (Principal component) ） $p$ ごとかつ細胞 $j$　ごとに主成分スコア $z_{j,p}$ が計算される
    - 例えば、 5番目の細胞の PC1 の主成分スコアは $z_{5,1}$
- 主成分スコア $z_{j,p}$　と各遺伝子 $i$ の発現量 $y^\prime_{i,j}$ の関連の強さを表す値を、因子負荷量 (factor loading) と呼ぶ。
    - 各主成分 $p$ と各遺伝子 $i$ の組み合わせについて、 因子負荷量 $w_{p,i}$ が計算できる
    - $w_{p,i}$ が正で絶対値が高い遺伝子は、主成分スコア $z_{j,p}$ が高い細胞で  $y^\prime_{i,j}$ の発現量が高いという関係がある
    - $w_{p,i}$ が負で絶対値が高い遺伝子は、主成分スコア $z_{j,p}$ が高い細胞で  $y^\prime_{i,j}$ の発現量が低いという関係がある


In [ ]:
# Command-23

# 遺伝子発現量行列に対してPCAを行う
# (PC_1, PC_2, などと表示されるはず)

planarian <- RunPCA(planarian, features = VariableFeatures(object = planarian))

In [ ]:
# Command-24

# 主成分（PC (Principal component) ）ごとに、その主成分スコアと遺伝子発現量の相関が高い（正負それぞれ）遺伝子5つを表示させる

print(planarian[["pca"]], dims = 1:5, nfeatures = 5)

In [ ]:
# Command-25

# 主成分（PC (Principal component) ）ごとに、その主成分スコアと遺伝子発現量の相関が高い（正負それぞれ）遺伝子を表示させる

VizDimLoadings(planarian, dims = 1:2, reduction = "pca")

In [ ]:
# Command-26

# 散布図を表示する
# 点は細胞
# PCAの結果から、各細胞 $j$ の PC1の主成分スコア $z_{j,1}$ （横軸）とPC2 の主成分スコア $z_{j,2}$ （縦軸）を表示している

DimPlot(planarian, reduction = "pca")

In [ ]:
# Command-27

# 主成分ごとにヒートマップを表示
# ヒートマップの横軸は細胞を表す。細胞は主成分スコアの順に並んでいる
# ヒートマップの縦軸は遺伝子。因子負荷量の絶対値が大きい遺伝子だけが表示されている
# 発現量に応じて各要素の色が異なる

DimHeatmap(planarian, dims = 1:9, cells = 500, balanced = TRUE)

## 11. 何番目の主成分までを考慮すべきかを調べる

- PCAでは、PC1, PC2, ...　の順に（元のデータが持つ変動の再構成に）重要となる
- 番号が後ろの主成分ほど、寄与が小さい（→無視できる）
- 何番目の主成分までを考慮すべきかを調べる


In [ ]:
# Command-28

# 準備

# 少し時間がかかります
planarian <- JackStraw(planarian, num.replicate = 100)
planarian <- ScoreJackStraw(planarian, dims = 1:20)

In [ ]:
# Command-29

# - 点線に近いほど重要でない

JackStrawPlot(planarian, dims = 1:15)

In [ ]:
# Command-30

# - 縦軸の値が小さいほど重要でない

ElbowPlot(planarian)

## 12. 細胞をクラスタリングする


- クラスタリングとは、たくさんのサンプルを、データの値の類似性に基づいて、いくつかのグループに分けることでさる
- １細胞RNA-seqの場合は、たくさんの細胞を、遺伝子発現量の類似性に基づいて、いくつかのグループに分けることである
- クラスタリングにより見つかったグループをクラスターと呼ぶ

In [ ]:
# Command-31

# クラスタリングの準備のために、各細胞について、似ている（＝類似度が大きい、近い）細胞を計算しておく

planarian <- FindNeighbors(planarian, dims = 1:10)

# クラスタリングを行う
# (Number of communities:  のところの数値が、見つかったクラスターの数)

planarian <- FindClusters(planarian, resolution = 0.5)

In [ ]:
# Command-32

# クラスタリングの結果（各細胞がどのクラスター（番号で表される）に割り当てられたか）を調べる
# 最初の１０細胞についてのみ表示

head(Idents(planarian), 10)

head(planarian$seurat_clusters, 10)

In [ ]:
# Command-33

#  各クラスターに割り当てられた細胞の数 (n_cell の列)を集計

as_tibble(Idents(planarian), rownames = "cell_barcode") %>% 
    rename(cluster=value) %>%
    group_by(cluster) %>%
    summarise(n_cell = n())

## B13: PCAの結果にさらにUMAPをかけて２次元空間に射影する

In [ ]:
# Command-34

# UMAPの計算を行う

planarian <- RunUMAP(planarian, dims = 1:10)

In [ ]:
# Command-35

# UMAP の計算結果を表示する
# 点が細胞
# 点の色はクラスターを表す

DimPlot(planarian, reduction = "umap")


## 14. 各クラスターに特徴的な遺伝子群を探す

In [ ]:
# Command-36

# クラスター 1 についてのマーカー遺伝子を探す

cluster1_markers <- FindMarkers(planarian, ident.1 = 1, min.pct = 0.25)

# マーカー遺伝子５つをとりあえず表示

head(cluster1_markers, n = 5)

In [ ]:
# Command-37

# クラスター5を クラスター0および3と分けるようなマーカー遺伝子を探す

cluster5_markers <- FindMarkers(planarian, ident.1 = 5, ident.2 = c(0, 3), min.pct = 0.25)
head(cluster5_markers, n = 5)

In [ ]:
# Command-38

# 全てのクラスターについて、あるクラスター　＄c$ とその他のクラスターを分けるようなマーカー遺伝子を探す
# クラスター $c$ で発現量が高く、他のクラスターでは発現量が低いようなものだけにする (`only.pos = TRUE` )

planarian_markers <- FindAllMarkers(planarian, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
# Command-39

# 各クラスターについて、avg_log2FC の値のトップ2遺伝子を表示
# (各クラスターのマーカー遺伝子を見ようとしている)

planarian_markers %>% 
    group_by(cluster) %>% 
    top_n(n = 2, avg_log2FC)

## 15. クラスターごとに遺伝子発現量をプロットする

In [ ]:
# Command-40

# Violin plot を表示
# 横軸はクラスターを表す
# 点は細胞
# 縦軸は遺伝子発現量を表す
# クラスター間で発現量の分布が異なることがわかる

VlnPlot(planarian, features = c("Smed-prog-2a-SmedASXL-014068-BPKG56961", "SmedASXL-008653"))

In [ ]:
# Command-41

# 各クラスターで avg_log2FC が高い遺伝子トップ5を抽出する

top5_markers <- planarian_markers %>% 
                    group_by(cluster) %>% 
                    top_n(n = 5, avg_log2FC)

options(repr.plot.width=15, repr.plot.height=10)

# ヒートマップを表示
# 横軸は細胞 (上に細胞が割り当てられているクラスターが表示されている)
# 縦軸は遺伝子
# 黄色い要素は発現量が高い
# 黄色く四角くまとまっている部分は、（先ほど見つけた）各クラスターでのみ発現量が高い遺伝子を表している

DoHeatmap(planarian, features = top5_markers$gene) + NoLegend()

## 16. 遺伝子発現量をUMAPの図に重ねる

In [ ]:
# Command-43

# UMAP の結果を表示
# 点が細胞
# 色がクラスターを表す

options(repr.plot.width=5, repr.plot.height=5)
DimPlot(planarian, reduction = "umap")

In [ ]:
# Command-44

# 各クラスターで avg_log2FC  が高いトップ1の遺伝子を抽出

planarian_markers %>% 
    group_by(cluster) %>% 
    top_n(n = 1, avg_log2FC) %>%
    .$gene -> each_cluster_features

print(each_cluster_features)

In [ ]:
# Command-45

# UMAP の図に、Command-43 で抽出した遺伝子を表示
# 点は細胞、色が高いのは各遺伝子の発現量が高いことを意味する
# Command-43 の図と比較し、各クラスターで発現量が高くなっているかを確かめる

options(repr.plot.width=15, repr.plot.height=10)

FeaturePlot(planarian, features = each_cluster_features)

In [ ]:
# Command-46

# マーカーについての情報をR dataset 形式で保存する

saveRDS(planarian_markers, file = "planarian_markers.rds")

# Seurat オブジェクトをR dataset 形式で保存する

saveRDS(planarian, file = "planarian_seurat.rds")